# Automatic Differentiation Explained 
### using Python with an application to linear regression solved using gradient descent

by <a href="mailto:carl.osipov@gmail.com"><b>Carl Osipov</b></a>, based on the materials from his <a href="https://bit.ly/mlops-book"><b>"MLOps Engineering at Scale"</b></a>.

In this notebook, you can learn about:

* what is a tensor in deep learning
* how are scalar and array tensors created and used in PyTorch
* how PyTorch `autograd` framework can be used for linear regression
* how reverse mode automatic differentiation can be implemented using Python
* the design principles behind PyTorch `autograd` application programming interface

# Tensor is the core deep learning data structure (container)

![title](https://github.com/osipov/odsc-europe-2022/raw/main/img/sbHDGMs.png)

* like in NumPy, a tensor is a multi-dimensional array. Unlike in NumPy, a tensor in deep learning supports differentiation
* the number of indices needed to access a value in the tensor is equal to the tensor dimension (rank)
* the number of values along every dimension (tensor shape) is constant, in other words, you can't have a matrix with 42 columns in the 1st row and 3 columns in the 2nd row (unlike "ragged" tensors).


### Create a `Scalar` class, a rank 0 tensor, with support for `__repr__` method.

In [70]:
class Scalar:    
    def __init__(self, val):
        self.val = float(val)
        self.grad = float(0)
        
    def __repr__(self):
        return f"Value: {self.val}, Gradient: {self.grad}"

### Check that you can create a `Scalar`

In [71]:
Scalar(42)

Value: 42.0, Gradient: 0.0

### Add support for `__add__`, and `__mul__` methods to the `Scalar` class.

In [72]:
class Scalar:    
    def __init__(self, val):
        self.val = float(val)
        self.grad = float(0)
        
    def __repr__(self):
        return f"Value: {self.val}, Gradient: {self.grad}"
    
    def __add__(self, other):
        result = Scalar(self.val + other.val)
        return result

    def __mul__(self, other):
        result = Scalar(self.val * other.val)
        return result        

### Check that you can do basic arithmetic with a `Scalar`

In [73]:
( Scalar(3) + Scalar(3) ) * Scalar(7)

Value: 42.0, Gradient: 0.0

### Create a placeholder function in `Scalar` to do autodiff

**NOTE:** Quick Python refresher `lambda: None` creates a function that returns `None`

In [74]:
def foo():
    return None

foo

<function __main__.foo()>

In [75]:
lambda: None

<function __main__.<lambda>()>

In [76]:
foo = lambda: None
foo

<function __main__.<lambda>()>

In [77]:
class Scalar:    
    def __init__(self, val):
        self.val = float(val)
        self.grad = float(0)
        
        #TODO: include a noop self.backward definition here
        self.backward = lambda: None
        
    def __repr__(self):
        return f"Value: {self.val}, Gradient: {self.grad}" 
    
    def __add__(self, other):
        result = Scalar(self.val + other.val)
        return result

    def __mul__(self, other):
        result = Scalar(self.val * other.val)        
        return result        

## Differentiation (finding derivatives) from first principles

**derivative:** a function specifying the slope of a line tangent to a target function for some value(s)

For instance, if the vertical dimension on the following coordinate plane is defined using $ y = f(x) $ where $ f(x) = x^2 $ (shown below)

we will use <a href="https://en.wikipedia.org/wiki/Leibniz%27s_notation">Leibniz's notation</a> $ \frac{\partial y}{\partial x} = 2x $ for the derivative of $ y $ with respect to $ x $ 

<br/>
<div>
    <a href="https://www.geogebra.org/classic/tfhqjsmw"><img src="https://github.com/osipov/odsc-europe-2022/raw/main/img/y-x-squared.png" width="600"/></a>
</div>




**Quick check:** Given $ y = f(x) = x $ what's the expression for $ \frac{\partial y}{\partial x} $. Hint below.

<div>
    <img src="https://github.com/osipov/odsc-europe-2022/raw/main/img/y-x.png" width="600"/>
</div>

## Understanding reverse mode accumulating automatic differentiation

Automatic differentiation (autodiff) is different from:

* **symbolic differentiation** taught in calculus classes or implemented in some computer math packages (e.g. Mathematica) that attempt to derive a general symbolic expression of the differentiated function for arbitrary values requiring more computation and memory than autodiff. 

* **numeric differentiation**, which is based on an approximation of $   \lim_{\epsilon \to 0} \frac{f(x + \epsilon) - f(x)}{\epsilon}$, which can be numerically unstable at the extreme values of the differentiated functions and  accumulate small errors introduced by floating point number approximations to real numbers.


Autodiff finds a **derivative of a function at specific values of the function's input variables** one at a time, with a computation complexity $ O(n) $ where `n` is the number of the mathematical operations used by the differentiated function. Notice that $ O(n) $ holds only when the number of outputs of the function is fewer than the number of inputs.

**NOTE:** Going forward I'll use gradient interchangably with derivative. The specific distinction has to do with calculation of derivatives of tensor variables vs. a derivative of a simple variable.


# Surprise!
### `Scalar` already does autodiff for a trivial (base) case. Let me show you.

### Create a `Scalar` instance for `x = 2.0`

In [78]:
x = Scalar(2)
x

Value: 2.0, Gradient: 0.0

### Define a function $ y $ using  `y = x` 

In [79]:
y = x
y

Value: 2.0, Gradient: 0.0

### Intuition

* **Given:** $ y = x $

<div>
    <img src="https://github.com/osipov/odsc-europe-2022/raw/main/img/y_eq_x.png" width="500"/>
</div>



### Perform autodiff on   `y = x` 
* Goal is to find $ \frac{\partial y}{ \partial x} $

### Step 1. Zero out the desired gradients 

**NOTE:** This is due to accumulation, explained shortly
* Use floating point values

Here the desired gradient $ \frac{\partial y}{ \partial x} $, so

In [80]:
x.grad = 0.
x.grad

0.0

### Step 2. Prepare for a call to reverse mode accumulating automatic differentiation via `backward`
* Use floating point values

How to initialize $ \frac{\partial y}{ \partial y} $ ?

In [81]:
y.grad = 1.

### Step 3. Call `backward` on `y`

* Compute an answer to the question: Given that  $ x $  changes by $ \partial x $, what's the ratio of a resulting change to `y` (here `dy`) to `dx`, or $ \frac{\partial y}{ \partial x} $

In [82]:
y.backward()

* check that $ \frac{\partial y}{ \partial x} = \frac{1}{1} = 1.0 $

### Step 4. Find out the gradient(s)

In [83]:
x.grad

1.0

### All together now

In [84]:
y = x
x.grad = 0.
y.grad = 1.
y.backward()
x.grad

1.0

* **Self-check:** Why the did the implementation return the correct answer?

# Implement autodiff for `Scalar` addition



### Intuition

* **Given:** $ z = c = a + b $


<div>
    <img src="https://github.com/osipov/odsc-europe-2022/raw/main/img/a_pl_b_eq_c_eq_z.png" width="500"/>
</div>



## Modify the` __add__` function to include  `backward` support
* **hint:** given $ y = x + z $, $ \frac{\partial y}{ \partial x} = 1.0 $ and $ \frac{\partial y}{ \partial z} = 1.0 $


* **hint:** don't forget the recursive `backward`

In [85]:
class Scalar:    
    def __init__(self, val):
        self.val = val
        self.grad = 0.
        self.backward = lambda: None
        
    def __repr__(self):
        return f"Value: {self.val}, Gradient: {self.grad}"
    
    def __add__(self, other):
        result = Scalar(self.val + other.val)

        def backward():        

          self.grad += result.grad
          other.grad += result.grad

          self.backward()
          other.backward()
            
        result.backward = backward
        
        return result

    def __mul__(self, other):
        result = Scalar(self.val * other.val)
        return result        

  ## Define `y = x + x + x` i.e. $y = 3 * x$

* evaluate for x = 42.

In [86]:
x = Scalar(42.)
y = (x + x) + x

## Prepare for and run the backward pass

In [87]:
x.grad = 0.
y.grad = 1.
y.backward()

* check that $ \frac{\partial y}{ \partial x} = 3.0 $

In [88]:
x.grad

3.0

* **Self-check:** Are you clear now why you needed accumulation `+=` in `backward`?

# Implement autodiff for `Scalar` multiplication


### Intuition

* **Given:** $ z = c = a * b $


<div>
    <img src="https://github.com/osipov/odsc-europe-2022/raw/main/img/a_ml_b_eq_c_eq_z.png" width="600"/>
</div>

## Implement `backward` support in the` __mul__` function


In [89]:
class Scalar:    
    def __init__(self, val):
        self.val = val
        self.grad = 0.
        self.backward = lambda: None
        
    def __repr__(self):
        return f"Value: {self.val}, Gradient: {self.grad}"
    
    def __add__(self, other):
        result = Scalar(self.val + other.val)
        def backward():        

          self.grad += result.grad
          other.grad += result.grad


          self.backward()
          other.backward()
            
        result.backward = backward
        return result

    def __mul__(self, other):
        result = Scalar(self.val * other.val)

        def backward():

          self.grad += other.val * result.grad
          other.grad += self.val * result.grad

          self.backward()
          other.backward()
        
        result.backward = backward

        return result        

## Use `y = x^3 + 2*x` for `x = 4.0`
* **hint:** recall that $ x^3 = x * x * x $
* **NOTE:** parentheses are just for clarity

In [90]:
x = Scalar(4.)

y = ( x * x * x ) + x + x 

* given $ y = x^3 + 2x $ the analytical solution to $ \frac{\partial y}{ \partial x} = 3x^2+2 $
* check that your implementation of `Scalar` returns the correct value of $ \frac{\partial y}{ \partial x} $ when $ x = 4.0 $

In [91]:
x.grad = 0.
y.grad = 1.
y.backward()

In [92]:
x.grad

50.0

## Apply `Scalar` to linear regression
* set the random seed to `42`
* set the epochs to `100`
* set the learning rate to `0.01`

In [93]:
SEED = 42
EPOCHS = 100
LEARNING_RATE = 0.01

import random
random.seed(a = SEED, version = 2)

* randomly init the model parameter `w` using uniform distribution [-1, 1]

In [94]:
w = Scalar(random.uniform(-1, 1))
w

Value: 0.2788535969157675, Gradient: 0.0

## Make linear regression data
* although the following is easier with NumPy, let's use pure Python to create a list of `Scalar` instances in a range `[-5,5)` with a step of `0.1`
* here's a reference scatter plot for what the data will look like

<div>
    <img src="img/linreg.png" width="600"/>
</div>

In [95]:
X = [Scalar(i * 0.1) for i in range(-50, 50)]
X[:5]

[Value: -5.0, Gradient: 0.0,
 Value: -4.9, Gradient: 0.0,
 Value: -4.800000000000001, Gradient: 0.0,
 Value: -4.7, Gradient: 0.0,
 Value: -4.6000000000000005, Gradient: 0.0]

In [96]:
W = Scalar(7)
y = [W * X[i] + Scalar(random.gauss(mu = 0, sigma = 1)) for i in range(len(X))]
y[:5]

[Value: -34.207855259050206, Gradient: 0.0,
 Value: -34.17448172429675, Gradient: 0.0,
 Value: -33.326400993441624, Gradient: 0.0,
 Value: -31.28992890553664, Gradient: 0.0,
 Value: -33.13805122143324, Gradient: 0.0]

## Implement a `forward` function using `w`
* **hint:** the function should return $ w * X $

In [97]:
def forward(X, w):
  y_pred = [w * X[i] for i in range(len(X))]
  return y_pred

## Implement the mean squared error calculation
* **hint:** Python `sum` can use a starter value as the 2nd argument

In [98]:
def mse(y, y_pred):
  error = [y[i] + Scalar(-1.0) * y_pred[i] for i in range(len(y))]
  se = [error[i] * error[i] for i in range(len(error))]
  mse = Scalar(1./len(error)) * sum(se, Scalar(0))
  return mse

## Confirm that gradient descent recovers the value used to generate `y` values.

In [99]:
for _ in range(EPOCHS):
  y_pred = forward(X, w)
  loss = mse(y, y_pred)

  w.grad = 0.
  loss.grad = 1.
  loss.backward()

  w.val -= LEARNING_RATE * w.grad
  
w

Value: 6.984187710500422, Gradient: -3.735900477863652e-14

Copyright 2022 CounterFactual.AI LLC. Licensed under GNU General Public License v3.0
Permissions of this strong copyleft license are conditioned on making available complete source code of licensed works and modifications, which include larger works using a licensed work, under the same license. Copyright and license notices must be preserved. Contributors provide an express grant of patent rights.